# Histogram-supported numeric range search

This one's kind of silly, but here goes.

We can really reduce the size of something like an index table to have simple counts matrix (or even yes/no) of bins of values, by data partition.

Using this, we can determine which pixels have values within particular ranges, restrict to only those pixels, then perform simple query within those pixels.

This requires the calculation of the 2d histogram ahead of time, but for this particular field, the pipeline takes around 2 minutes to run, and the histogram is re-usable as many times as you need it.

In [1]:
import lsdb
import numpy as np
import pandas as pd
from lsdb.core.search.abstract_search import AbstractSearch
from lsdb.types import HCCatalogTypeVar
import nested_pandas as npd
from hats import HealpixPixel

In [2]:
gaia_d = lsdb.open_catalog("/epyc/data3/hats/catalogs/gaia_edr3_distances/gaia_edr3_distances")

In [3]:
log_bins = np.logspace(np.log10(1), np.log10(80_000), num=1000)

In [4]:
class HistogramdSearch(AbstractSearch):
    
    def __init__(self, histogrammy_parquet_file, bins,
                 greater_than_value = None, less_than_value = None, 
                 inclusive=False, fine:bool=True):
        super().__init__(fine)
        if less_than_value is None or greater_than_value is None:
            raise ValueError("Both of less_than_value or greater_than_value are required (for now)")
        if greater_than_value > less_than_value:
            raise ValueError("less_than_value should be less than greater_than_value")
        if histogrammy_parquet_file is None or bins is None:
            raise ValueError("histogrammy_parquet_file and bins are required")
        self.less_than_value = less_than_value
        self.greater_than_value = greater_than_value
        self.bins = bins
        frame = pd.read_parquet(histogrammy_parquet_file)
        self.hist_2d = np.stack(frame["log_binned_histogram"].values)
        self.pixels = np.array([HealpixPixel(order, pixel)
            for order, pixel in zip(
                frame["Norder"],
                frame["Npix"],
            )])
    
    def perform_hc_catalog_filter(self, hc_structure: HCCatalogTypeVar) -> HCCatalogTypeVar:
        """Determine the pixels for which there is a result in each field"""
        bin_start = np.searchsorted(self.bins, self.greater_than_value, side='left')
        bin_end = np.searchsorted(self.bins, self.less_than_value, side='right')
        print("bins", bin_start, bin_end)
        squish = self.hist_2d[:,bin_start:bin_end].sum(axis=1)
        limit_pixels = self.pixels[np.nonzero(squish)]
        if len(limit_pixels) == 0:
            raise ValueError("Error with extrema search - there's nothing that extreme!!")
        return hc_structure.filter_from_pixel_list(limit_pixels) 
    
    def search_points(self, frame: npd.NestedFrame, _) -> npd.NestedFrame:
        """Determine the search results within a data frame"""
        return frame.query(f'{self.greater_than_value} < r_med_photogeo < {self.less_than_value}')

In [5]:
%%time
search_object = HistogramdSearch("hist_grid", log_bins,  greater_than_value=1, less_than_value=3)
gaia_d.search(search_object).compute()

bins 0 98
bins 0 98
CPU times: user 4.52 s, sys: 4.84 s, total: 9.35 s
Wall time: 2.94 s


,source_id,r_med_geo,r_lo_geo,r_hi_geo,r_med_photogeo,r_lo_photogeo,r_hi_photogeo,flag,ra,dec,Norder,Dir,Npix
_healpix_29,,,,,,,,,,,,,
381407739701909405,762815470562110464,2.545952,2.545706,2.546205,2.545851,2.545692,2.546106,10033,165.83096,35.948653,2,0,21
1473525239409412345,2947050466531873024,2.670147,2.668469,2.671663,2.670632,2.668852,2.672157,10012,101.286626,-16.720933,4,0,1308
1932486467174911786,3864972938605115520,2.408358,2.407998,2.408747,2.408285,2.407773,2.40883,10022,164.10319,7.002727,2,0,107
2037570889377880190,4075141768785646848,2.975431,2.975131,2.975705,2.975421,2.975177,2.975699,10033,282.458789,-23.837097,5,0,7238
2236416425020061922,4472832130942575872,1.828106,1.827975,1.828225,1.8281,1.82797,1.828225,10033,269.448503,4.73942,4,0,1986
2570346799484574619,5140693571158739840,2.718584,2.713234,2.724433,2.718944,2.714043,2.724843,10022,24.771554,-17.9483,2,0,142
2570346800245147123,5140693571158946048,2.67459,2.67066,2.678184,2.675949,2.672405,2.679119,10022,24.771674,-17.947683,2,0,142
2926749359394994434,5853498713190525696,1.301911,1.301814,1.302005,1.301935,1.301849,1.302006,10033,217.392321,-62.676075,6,40000,41591


## Appendix - construction of the histogram

I played around with the number of bins for this particular field. I wanted the resulting histogram to be pretty sparse - if there's something in every bin, then the restriction isn't helpful. Unfortunately, distance is a typical distribution, so this is not the ideal kind of field. But this is a proof of concept, ok?

In [ ]:
log_bins = np.logspace(np.log10(1), np.log10(80_000), num=1000)
log_bins

In [6]:
def make_hists(df, pixel):
    histy, bin_edges = np.histogram(df['r_med_photogeo'].dropna().values, bins=log_bins)
    return pd.DataFrame(
        [
            {
                "pixel": pixel,
                "Norder": pixel.order,
                "Npix": pixel.pixel,
                "log_binned_histogram": histy,
            }
        ]
    )

In [7]:
unrealized = gaia_d.map_partitions(make_hists, include_pixel=True)

In [8]:
%%time
realized = unrealized.compute()

CPU times: user 14min 35s, sys: 6min 10s, total: 20min 46s
Wall time: 1min 46s


In [9]:
realized["Norder", "Npix", "log_binned_histogram"].to_parquet("hist_grid")